In [4]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# STEP 1: Organize your dataset into a flat folder with labels as subfolders
def copy_images(source_root, dest_root):
    if not os.path.exists(dest_root):
        os.makedirs(dest_root)

    for crop_type in os.listdir(source_root):
        crop_path = os.path.join(source_root, crop_type)
        if not os.path.isdir(crop_path):
            continue

        for root, dirs, files in os.walk(crop_path):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    label = os.path.basename(root)
                    dest_label_path = os.path.join(dest_root, label)
                    os.makedirs(dest_label_path, exist_ok=True)
                    src_file = os.path.join(root, file)
                    dst_file = os.path.join(dest_label_path, file)
                    shutil.copy2(src_file, dst_file)

# Modify these paths as needed
source_root = r"C:\Users\RamaKrishna\OneDrive\Documents\crop dataset"
flat_dataset_path = r"C:\Users\RamaKrishna\OneDrive\Documents\flat_crop_dataset"

copy_images(source_root, flat_dataset_path)

# STEP 2: Load images using ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    flat_dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    flat_dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# STEP 3: Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# STEP 4: Train the model
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen
)

# STEP 5: Evaluate the model
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_accuracy:.2f}")


Found 26488 images belonging to 29 classes.
Found 6610 images belonging to 29 classes.


c:\Users\RamaKrishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


c:\Users\RamaKrishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


828/828 ━━━━━━━━━━━━━━━━━━━━ 466s 559ms/step - accuracy: 0.3038 - loss: 2.1087 - val_accuracy: 0.4454 - val_loss: 1.4964
Epoch 2/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 648s 780ms/step - accuracy: 0.4421 - loss: 1.4748 - val_accuracy: 0.4637 - val_loss: 1.4216
Epoch 3/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 573s 690ms/step - accuracy: 0.4882 - loss: 1.3169 - val_accuracy: 0.5027 - val_loss: 1.3748
Epoch 4/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 461s 554ms/step - accuracy: 0.5193 - loss: 1.2260 - val_accuracy: 0.5011 - val_loss: 1.3792
Epoch 5/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 410s 494ms/step - accuracy: 0.5466 - loss: 1.1540 - val_accuracy: 0.4950 - val_loss: 1.5229
Epoch 6/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 413s 496ms/step - accuracy: 0.5596 - loss: 1.1027 - val_accuracy: 0.5172 - val_loss: 1.3536
Epoch 7/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 421s 506ms/step - accuracy: 0.5827 - loss: 1.0398 - val_accuracy: 0.5374 - val_loss: 1.4026
Epoch 8/10
828/828 ━━━━━━━━━━━━━━━━━━━━ 411s 494ms/step - accuracy: 0.6094 - loss: 0.98

In [5]:
model.save("crop_disease_model.h5")
